In [17]:
import sentiment.model as funcs
import os
import multiprocessing as mp
import numpy as np
from sentiment.model import time_to_complete
import multiprocessing as mp
from gensim.models import Word2Vec
from data_handler.models import Word
from sentiment.models import Category

os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
def progress(count, total, status=''):
    bar_len = 60
    filled_len = int(round(bar_len * count / float(total)))

    percents = round(100.0 * count / float(total), 1)
    bar = '=' * filled_len + '-' * (bar_len - filled_len)

    sys.stdout.write('[%s] %s%s ...%s\r' % (bar, percents, '%', status))
    sys.stdout.flush()


In [14]:
import pandas as pd
df = pd.read_excel('lmd.xlsx', sheet_name='LMD_Positive', header=None, names=["Entry"])
df

,Entry
0,ABANDON
1,ABANDONED
2,ABANDONING
3,ABANDONMENT
4,ABANDONMENTS
...,...
2350,WRONGDOING
2351,WRONGDOINGS
2352,WRONGFUL
2353,WRONGFULLY


In [19]:
df = df.replace(np.nan, '', regex=True)
cat = Category.objects.create(name="LMD_Negative")
length = len(df)
for index, row in df.iterrows():
    word = str(row["Entry"]).strip()
    
    progress(index, length, status="Adding word: {}".format(word))
    word_obj = ""
    word_obj = Word.objects.filter(word=word).first()
    if not word_obj:
        word_obj = Word(word=word)
        word_obj.save()
    word_obj.categories.add(cat)
    word_obj.save()

In [ ]:
def similarity(wv, matrix, word_list, word):
    matrix[word] = [wv.similarity(word, other_word) for other_word in word_list]
    return matrix

In [ ]:
index = 0
length = len(word_list)
a_matrix = {}
for word in word_list:
    index += 1
    progress(index, length, status="Processed word {}/{}. Len: {}".format(index, length, len(a_matrix)))
    a_matrix = pool.apply(similarity, args=(wv, a_matrix, word_list, word))
    


In [ ]:
word = word_list[0]
print(word)

a_matrix = pool.apply(similarity, args=(wv, a_matrix, word_list, word))
print(a_matrix)

In [ ]:
import pandas as pd
from data_handler.models import StockPrice
df = pd.read_csv("data_handler/GBPEUR=X.csv")

In [ ]:
df.columns

In [ ]:
from data_handler.models import Asset

asset = Asset.objects.create(name="GBP/USD", ticker="GBPUSD=X")
asset.save()

In [ ]:
df2 = pd.read_csv("data_handler/GBPUSD=X.csv")
import math
from datetime import datetime 
length, _ = df2.shape
df2 = df2.dropna()
for index, row in df2.iterrows():
    progress(index, length)
    
    date = datetime.strptime(row['Date'], "%Y-%m-%d").date()
    
    stock = StockPrice.objects.create(asset=asset, 
                                      date=date, 
                                      open=row['Open'], 
                                      close=row['Close'], 
                                      high=row['High'], 
                                      low=row['Low'],
                                      adj_close=row['Adj Close'] if row['Adj Close'] else 0.0,
                                      volume=row['Volume'] if not math.isnan(row['Volume']) else 0.0,
                                      interday_volatility=0.0
                                     )
    stock.save()
    stock.interday_volatility = stock.log_return()
    stock.save()
    